The computation cell is illustrated below---a rectangular domain with PML boundaries and a small design region at the center.

<img src="ComputationCell.png" alt="Drawing" style="width: 600px;"/>

In [39]:
using Gmsh
using Gridap
using GridapGmsh
using SparseArrays
using Gridap.Geometry
using ChainRules
using Zygote
using NLopt
import Gmsh: gmsh
import ChainRules: rrule

Gridap.outer(a::Number,b::Number) = a * b
Gridap.Helpers.operate(::typeof(tanh),x::Float64)=tanh(x)

include("MeshGenerator.jl")
include("PML.jl")
include("Helper.jl")
include("FilterAndThreshold.jl")

# Physical parameters 
λ = 1.0          # Wavelength (arbitrary unit)
k = 2*π/λ        # Wave number 
ω = k            # c=1
ϵ_1 = 1.0        # Relative electric permittivity for material 1
ϵ_2 = 3.0        # Relative electric permittivity for material 2
μ = 1.0          # Relative magnetic permeability for all materials

# Geometry parameters of the mesh
L = 4.0          # Length of the normal region
H = 4.0          # Height of the normal region
d_pml = 0.8      # Thickness of the PML
L_d = L/2        # Length of the design region
H_d = H/5        # Height of the design region
r_t = L/40       # Radius of the target circle
y_t = -(H_d+H)/4 # y-position of the target circle (x fixed to 0)
LH = [L,H]

# Characteristic length (controls the resolution, smaller the finer)
resol = 10       # Number of points per wavelength
l_0 = λ/resol    # Normal region
l_d = l_0/5      # Design region
l_pml = 2*l_0    # PML 

# Point source location
pos = [0.0,H/2.0*0.9]
δ = λ/resol      # Gaussian point source width
I = 1e4

# PML parameters
R = 1e-4         # Tolerence for PML reflection 
σ = -3/4*log(R)/d_pml/sqrt(ϵ_1)

# Generate mesh
MeshGenerator(L,H,L_d,H_d,r_t,y_t,d_pml,l_0,l_d,l_pml)
include("GridapSetup.jl")
include("Objective.jl")

Info    : No current model available: creating one
Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Meshing curve 2 (Line)
Info    : Meshing curve 3 (Line)
Info    : Meshing curve 4 (Line)
Info    : Meshing curve 5 (Line)
Info    : Meshing curve 6 (Line)
Info    : Meshing curve 7 (Line)
Info    : Meshing curve 8 (Line)
Info    : Meshing curve 9 (Line)
Info    : Meshing curve 10 (Line)
Info    : Meshing curve 11 (Line)
Info    : Meshing curve 12 (Line)
Info    : Meshing curve 13 (Line)
Info    : Meshing curve 14 (Line)
Info    : Meshing curve 15 (Line)
Info    : Meshing curve 16 (Line)
Info    : Meshing curve 17 (Line)
Info    : Meshing curve 18 (Line)
Info    : Meshing curve 19 (Line)
Info    : Meshing curve 20 (Line)
Info    : Meshing curve 21 (Line)
Info    : Meshing curve 22 (Line)
Info    : Meshing curve 23 (Line)
Info    : Meshing curve 24 (Line)
Info    : Meshing curve 25 (Li

g_p (generic function with 2 methods)

In [43]:
# Filter and threshold paramters
r = l_d*1.0      # Filter radius
β = 10.0          # β∈[1,∞], threshold sharpness
η = 0.5          # η∈[0,1], threshold center

# Loss control
α = 0.

opt = Opt(:LD_MMA, np)
opt.lower_bounds = 0.0
opt.upper_bounds = 1.0
opt.ftol_rel = 1e-3
opt.maxeval = 500
opt.max_objective = g_p

#(g_opt,p_opt,ret) = optimize(opt, rand(np))
(g_opt,p_opt,ret) = optimize(opt, p)
numevals = opt.numevals # the number of function evaluations

# Display u and ε
p = p_opt
pf = pf_p(p)
uvec = u_pf(pf)
ϵ0 = ϵ_1 + (ϵ_2-ϵ_1)*FEFunction(P,p_vec(p,P,tags,design_tag))
ϵt = ϵ_1 + (ϵ_2-ϵ_1)*Threshold(β,η,FEFunction(Pf,pf))
writevtk(trian,"demo",cellfields=["ϵ0"=>ϵ0,"ϵt"=>ϵt,"Real"=>FEFunction(U,real(uvec)),"imag"=>FEFunction(U,imag(uvec)),"Norm"=>FEFunction(U,sqrt.(real(uvec).^2+imag(uvec).^2))])


g_value = g_p(p) = 21250.6650438623
g_value = g_p(p) = 20510.533270764383
g_value = g_p(p) = 21508.076530127437
g_value = g_p(p) = 21969.711774371925
g_value = g_p(p) = 22109.98413390885
g_value = g_p(p) = 22141.068538867148
g_value = g_p(p) = 22153.60620989059
g_value = g_p(p) = 22159.13404732793
g_value = g_p(p) = 22161.66153121595
g_value = g_p(p) = 22162.83985327816
g_value = g_p(p) = 22163.39316494492
g_value = g_p(p) = 22163.653851369836
g_value = g_p(p) = 22163.77686080061
g_value = g_p(p) = 22163.83494876188
g_value = g_p(p) = 22163.86238887157
g_value = g_p(p) = 22163.87535341963
g_value = g_p(p) = 22163.881479220287
g_value = g_p(p) = 22163.88437379248
g_value = g_p(p) = 22163.885741563707
g_value = g_p(p) = 22163.886387881616
g_value = g_p(p) = 22163.886693289798
g_value = g_p(p) = 22163.886837606162
g_value = g_p(p) = 22163.886905801002
g_value = g_p(p) = 22163.886938025516
g_value = g_p(p) = 22163.886953252913
g_value = g_p(p) = 22163.886960448315
g_value = g_p(p) = 22163.

1-element Array{String,1}:
 "demo.vtu"

In [26]:
using Richardson
p0 = zeros(np)
δp = rand(np)
extrapolate(δp*0.1, rtol=0) do h
    @show norm(h)
    (g_p(h)-g_p(p0)) / norm(h)
end

norm(h) = 5.526855856115665
norm(h) = 0.6908569820144581
norm(h) = 0.08635712275180726
norm(h) = 0.010794640343975908
norm(h) = 0.0013493300429969885
norm(h) = 0.00016866625537462356


(-2.659018663252036, 2.4552111455022896e-9)

In [27]:
dgdp=zeros(np)
g_p(p0,dgdp)
dgdp'*δp/norm(δp)

g_value = g_p(p) = 974.197420080274


-2.6204661897568906

In [12]:
g_p(rand(np))

2.8485431038251677e-5